In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import itertools

In [ ]:
# All possible data
def binary_sequences(length):
    return np.array([[int(i) for i in np.binary_repr(j, width=length)] for j in range(2**length)])
all_X = binary_sequences(4)

# Definition of models
def llh_factorised(x):
    c = x[0]
    z1 = x[1]
    z2 = x[2]
    r = x[3]
    if c == 0:
        if z1 == r:
            return 1
        else:
            return 0
    elif c == 1:
        if z2 == r:
            return 1
        else:
            return 0
    else:
        print('error')
        return None

def llh_x(x):
    c = x[0]
    z1 = x[1]
    z2 = x[2]
    r = x[3]
    if z1 == r:
        return 1
    else:
        return 0

def llh_x_noisy(x):
    c = x[0]
    z1 = x[1]
    z2 = x[2]
    r = x[3]
    noise = 0.2
    if z1 == r:
        return 1 - noise
    else:
        return noise

def llh_y(x):
    c = x[0]
    z1 = x[1]
    z2 = x[2]
    r = x[3]
    if z2 == r:
        return 1
    else:
        return 0

def llh_2D(x, parameter_list=[1,1]):
    c = x[0]
    z1 = x[1]
    z2 = x[2]
    r = x[3]
    z1r = parameter_list[0]
    z2r = parameter_list[1]
    if z1 == z1r and z2 == z2r:
        if r == 1:
            return 1
        else:
            return 0
    else:
        if r == 0:
            return 1
        else:
            return 0

def llh_2D_noisy(x,noise=0.2):
    c = x[0]
    z1 = x[1]
    z2 = x[2]
    r = x[3]
    if z1 and z2:
        if r == 1:
            return 1 - noise
        return 1 - noise
    else:
        return noise

def llh_factorised_noisy(x,parameter_list = [0.2,0.2]):
    c = x[0]
    z1 = x[1]
    z2 = x[2]
    r = x[3]
    noise1 = parameter_list[0]
    noise2 = parameter_list[1]
    if c == 0:
        if z1 == r:
            return 1 - noise1
        else:
            return noise1
    elif c == 1:
        if z2 == r:
            return 1 - noise2
        else:
            return noise2
    else:
        print('error')
        return None
        

def PX_array(model):
    '''Return PXas a single array given a model'''
    PX = np.zeros((16,5))
    for i in range(16):
        PX[i,0:4] = all_X[i]
        PX[i,4] = model(all_X[i])
    PX[:,4] = PX[:,4]/np.sum(PX[:,4])
    return PX

def R_from_PX(PX, ctx):
    '''Return reward matrix from PX given a context'''
    max_PX = np.max(PX[:,4])
    zr = PX[np.where((PX[:,0] == ctx) & (PX[:,4] == max_PX))[0], 1:4]
    R = np.zeros((2,2))
    for i in range(2):
        for j in range(2):
            R[i,j] = zr[np.where((zr[:,0] == i) & (zr[:,1] == j))[0], 2]
    return R

def R_plot(PX):
    '''Plot R matrices in both contexts'''
    fig, ax = plt.subplots(1,2)
    ax[0].imshow(R_from_PX(PX, 0), interpolation='nearest')
    ax[0].set_title('Context 0')
    ax[1].imshow(R_from_PX(PX, 1), interpolation='nearest')
    ax[1].set_title('Context 1')
    for i in range(2):
        ax[i].set_xticks([])
        ax[i].set_yticks([])
    plt.show()

def sigmoid(x):
    return 1 / (1 + np.exp(-x-2))

def binarised(PX):
    PX_bin = np.zeros((16,5))
    PX_bin[:,0:4] = PX[:,0:4]
    PX_bin[:,4] = np.where(PX[:,4] > 0., 1., 0.)
    return PX_bin

def logp(PX):
    PX_log = np.zeros((16,5))
    PX_log[:,0:4] = PX[:,0:4]
    PX_log[:,4] = sigmoid(np.log(PX[:,4]))
    return PX_log

def PX_plot(PX):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(5,2))
    ax1.imshow(PX, cmap='gray', interpolation='nearest')
    ax1.set_xticks(np.arange(5))
    ax1.set_xticklabels(['c', 'z1', 'z2', 'r', 'p'])
    ax1.set_yticks([])
    ax2.bar(np.arange(PX.shape[0]), PX[:,4])
    ax2.set_xticks(np.arange(16))
    ax2.set_xticklabels([str(x).replace('.','').replace('[','').replace(']','') for x in PX[:,:4]], rotation='vertical')
    ax2.set_ylabel('P(x)')
    # ax2.set_ylim(0,1)
    box = ax2.get_position()
    ax2.set_position([box.x0, box.y0, box.width * 2, box.height])
    plt.show()

def sorted(PX):
    '''Return sorted PX'''
    return PX[PX[:,4].argsort()[::-1]]


In [ ]:
PX = PX_array(llh_factorised)

In [ ]:
plt.imshow(PX,cmap='gray', interpolation='nearest')
plt.xticks(np.arange(5), ['c', 'z1', 'z2', 'r', 'p'])
plt.yticks([])
plt.show()

In [ ]:
def X_plot(x):
    '''Plot X vectors in matrix '''
    c = x[0]
    z1 = x[1]
    z2 = x[2]
    r = x[3]
    R = np.empty((2,2,2))
    R[:] = np.nan
    R[c,z1,z2] = r
    fig, ax = plt.subplots(1,2)
    ax[0].imshow(R[0], interpolation='nearest', vmin=0, vmax=1)
    #ax[0].set_title('ctx 0',fontsize=6)
    ax[1].imshow(R[1], interpolation='nearest', vmin=0, vmax=1)
    #ax[1].set_title('ctx 1',fontsize=6)
    for i in range(2):
        ax[i].set_xticks([])
        ax[i].set_yticks([])
    plt.gcf().set_size_inches((1,.5))
    plt.show()

In [ ]:
#same X_plot but with a third subplot to the right that contains text
def X_plot_text(x):
    '''Plot X vectors in matrix '''
    c = x[0]
    z1 = x[1]
    z2 = x[2]
    r = x[3]
    R = np.empty((2,2,2))
    R[:] = np.nan
    R[c,z1,z2] = r
    fig, ax = plt.subplots(1,3)
    ax[0].imshow(R[0], interpolation='nearest', vmin=0, vmax=1)
    #ax[0].set_title('ctx 0',fontsize=6)
    ax[1].imshow(R[1], interpolation='nearest', vmin=0, vmax=1)
    #ax[1].set_title('ctx 1',fontsize=6)
    for i in range(2):
        ax[i].set_xticks([])
        ax[i].set_yticks([])
    ax[2].axis('off')
    ax[2].text(0.3,0.4,str(x),fontsize=10)
    plt.gcf().set_size_inches((2,1))
    plt.show()

In [ ]:
for i in range(8):
    X_plot_text(all_X[i])

In [ ]:
plt.bar(np.arange(16), PX[:,4])
plt.xticks(np.arange(16), [str(x).replace('.','').replace('[','').replace(']','') for x in PX[:,:4]], rotation='vertical')
plt.ylabel('P(x)')
plt.ylim(0,1)
fig = plt.gcf()
fig.set_size_inches(15,4)
plt.show()

In [ ]:
PX = PX_array(llh_factorised)
PX_plot(PX)
R_plot(PX)

In [ ]:
PX = PX_array(llh_x)
PX_plot(PX)
R_plot(PX)

In [ ]:
PX = PX_array(llh_x_noisy)
PX_plot(PX)

In [ ]:
PX = PX_array(llh_x)
PX_plot(PX)

In [ ]:
PX = PX_array(llh_factorised_noisy)
PX_plot(PX)

In [ ]:
PX = PX_array(llh_factorised_noisy)
PX_plot(PX)

In [ ]:
PX = PX_array(lambda x: llh_factorised_noisy(x, [0.2, 0.4]))
PX_plot(PX)

In [ ]:
PX = PX_array(lambda x: llh_factorised_noisy(x, [0.2, 0.2]))
PX_plot(PX)
PX_plot(sorted(PX))

In [ ]:
PX = PX_array(llh_2D)
PX_plot(PX)

In [ ]:
N = 1000
thetasamples = np.random.binomial(1, 0.5, (N,2))
#create PX arra np.zeros
PX = np.zeros((16,5))
for j in range(N):
    PX += PX_array(lambda x: llh_2D(x, thetasamples[j]))
PX = PX/N
PX_plot(PX)
PX_1x2D_marginalised = PX

In [ ]:
plt.plot(np.arange(16), PX_array(llh_factorised)[:,4])
plt.plot(np.arange(16), PX_array(llh_x)[:,4])
plt.plot(np.arange(16), PX_array(llh_x_noisy)[:,4])
plt.plot(np.arange(16), PX_array(llh_y)[:,4])
plt.plot(np.arange(16), PX_array(llh_factorised_noisy)[:,4])
plt.plot(np.arange(16), PX_array(lambda x: llh_factorised_noisy(x, [0.2, 0.4]))[:,4])
plt.plot(np.arange(16), PX_1x2D_marginalised[:,4])
plt.xticks(np.arange(16), [str(x).replace('.','').replace('[','').replace(']','') for x in PX[:,:4]], rotation='vertical')
plt.ylabel('P(x)')
plt.legend(['factorised', 'x', 'noisy x', 'y', 'factorised noisy', 'factorised noisy (0.2, 0.4)', '1x2D'], bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
fig = plt.gcf()
fig.set_size_inches(10,4)
plt.show()

In [ ]:
plt.plot(np.arange(16), sorted(PX_array(llh_factorised))[:,4])
plt.plot(np.arange(16), sorted(PX_array(llh_x))[:,4])
plt.plot(np.arange(16), sorted(PX_array(llh_x_noisy))[:,4])
plt.plot(np.arange(16), sorted(PX_array(llh_y))[:,4])
plt.plot(np.arange(16), sorted(PX_array(llh_factorised_noisy))[:,4])
plt.plot(np.arange(16), sorted(PX_array(lambda x: llh_factorised_noisy(x, [0.2, 0.4])))[:,4])
plt.plot(np.arange(16), sorted(PX_1x2D_marginalised)[:,4])
plt.ylabel('P(x)')
fig = plt.gcf()
fig.set_size_inches(10,4)
plt.xticks([])
plt.xlabel('x (sorted by P(x), varies between models)')
plt.legend(['factorised', 'x', 'noisy x', 'y', 'factorised noisy', 'factorised noisy (0.2, 0.4)', '1x2D'])
plt.show()

In [ ]:
plt.plot(np.arange(16), PX_array(llh_factorised)[:,4])
plt.plot(np.arange(16), PX_array(llh_x)[:,4])
plt.xticks(np.arange(16), [str(x).replace('.','').replace('[','').replace(']','') for x in PX[:,:4]], rotation='vertical')
plt.ylabel('P(x)')
# legend
plt.legend(['factorised', 'x'], bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
fig = plt.gcf()
fig.set_size_inches(10,4)
plt.show()

In [ ]:
plt.plot(np.arange(16), PX_array(llh_x_noisy)[:,4])
plt.plot(np.arange(16), PX_array(llh_factorised_noisy)[:,4])
plt.xticks(np.arange(16), [str(x).replace('.','').replace('[','').replace(']','') for x in PX[:,:4]], rotation='vertical')
plt.ylabel('P(x)')
# legend
plt.legend(['noisy x', 'factorised noisy'], bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
fig = plt.gcf()
fig.set_size_inches(10,4)
plt.show()

In [ ]:
# Generate all possible datasets of size 3
# all possible triple combinations of all_X
all_X_permut3 = np.array(list(itertools.product(all_X, all_X, all_X)))
# concatenate all triple combinations
D_of_size3 = np.array([np.concatenate(x) for x in all_X_permut3])
# encoded as integers instead of binary
D_of_size3_ints = np.array(list(itertools.product(np.arange(16), np.arange(16), np.arange(16))))

In [ ]:
# generate samples from prior
thetasamples = np.random.beta(4, 10, 100)

# initialise array for mllhs
mllhs = np.zeros((D_of_size3_ints.shape[0],len(thetasamples)))

# for each dataset (i) and for each sample from the prior (j), compute the likelihoods
for i in range(D_of_size3_ints.shape[0]):    
    for j in range(len(thetasamples)):
        mllhs[i,j] = llh_factorised_noisy(all_X[D_of_size3_ints[i,0]],[thetasamples[j]]) * \
                                llh_factorised_noisy(all_X[D_of_size3_ints[i,1]],[thetasamples[j]]) * \
                                llh_factorised_noisy(all_X[D_of_size3_ints[i,2]],[thetasamples[j]])
# average over samples from the prior
mllhs = np.mean(mllhs,axis=1)

In [ ]:
mllhs_factorised = np.zeros(D_of_size3_ints.shape[0])
for i in range(D_of_size3_ints.shape[0]):
    mllhs_factorised[i] = llh_factorised(all_X[D_of_size3_ints[i,0]]) * llh_factorised(all_X[D_of_size3_ints[i,1]]) * llh_factorised(all_X[D_of_size3_ints[i,2]])

In [ ]:
# sorted llhs
plt.plot(range(D_of_size3_ints.shape[0]),np.sort(mllhs)[::-1])
plt.plot(range(D_of_size3_ints.shape[0]),np.sort(mllhs_factorised)[::-1])
fig = plt.gcf()
fig.set_size_inches(15,4)
# legends
plt.legend(['factorised noisy marginalised', 'factorised deterministic'])
plt.ylabel('mllh')
plt.xlabel('x (sorted by P(x), varies between models)')
plt.show()

In [ ]:
# sorted llhs
plt.plot(range(300), mllhs[:300])
plt.plot(range(300), mllhs_factorised[:300])
fig = plt.gcf()
fig.set_size_inches(20,4)
# legends
plt.legend(['factorised noisy marginalised', 'factorised deterministic'])
plt.ylabel('mllh')
plt.xlabel('x (sorted by P(x), varies between models)')
plt.show()

In [ ]:
# Generate all possible datasets of size 2
# all possible pair combinations of all_X
all_X_permut = np.array(list(itertools.product(all_X, all_X)))
# concatenate all pairs
D_of_size2 = np.array([np.concatenate(x) for x in all_X_permut])
# encoded as integers instead of binary, i.e. want to index by n-th possible dataset etc.
D_of_size2_ints = np.array(list(itertools.product(np.arange(16), np.arange(16))))

plt.imshow(D_of_size2.T,cmap='gray')
# set figsize
fig = plt.gcf()
fig.set_size_inches(20,4)
# hide ticks
plt.xticks([])
plt.yticks([]);

In [ ]:
def llh_factorised_noisy(x,parameter_list = [0.2]):
    c = x[0]
    z1 = x[1]
    z2 = x[2]
    r = x[3]
    noise = parameter_list[0]
    if c == 0:
        if z1 == r:
            return 1 - noise
        else:
            return noise
    elif c == 1:
        if z2 == r:
            return 1 - noise
        else:
            return noise
    else:
        print('error')
        return None

In [ ]:
llhs = np.zeros(256)
for i in range(256):
    llhs[i] = llh_factorised(all_X[D_of_size2_ints[i,0]]) * llh_factorised(all_X[D_of_size2_ints[i,1]])

In [ ]:
# generate samples from prior
thetasamples = np.random.beta(4, 10, 100)

# initialise array for mllhs
mllhs = np.zeros((D_of_size2_ints.shape[0],len(thetasamples)))

# for each dataset (i) and for each sample from the prior (j), compute the likelihoods
for i in range(D_of_size2_ints.shape[0]):    
    for j in range(len(thetasamples)):
        mllhs[i,j] = llh_factorised_noisy(all_X[D_of_size2_ints[i,0]],[thetasamples[j]]) * \
                                llh_factorised_noisy(all_X[D_of_size2_ints[i,1]],[thetasamples[j]])
# average over samples from the prior
mllhs = np.mean(mllhs,axis=1)

In [ ]:
# generate samples from prior
#random binary vector of length 2
thetasamples = np.random.binomial(1, 0.5, (100,2))

# initialise array for mllhs
mllhs_1x2D = np.zeros((D_of_size2_ints.shape[0],len(thetasamples)))

# for each dataset (i) and for each sample from the prior (j), compute the likelihoods
for i in range(D_of_size2_ints.shape[0]):    
    for j in range(len(thetasamples)):
        mllhs_1x2D[i,j] = llh_2D(all_X[D_of_size2_ints[i,0]],thetasamples[j]) * \
                                llh_2D(all_X[D_of_size2_ints[i,1]],thetasamples[j])
# average over samples from the prior
mllhs_1x2D = np.mean(mllhs_1x2D,axis=1)

In [ ]:
D_of_size2_ints[i,1]

In [ ]:
# visualise theta prior
plt.hist(thetasamples)

In [ ]:
mllhs_noisy = np.zeros(256)
for i in range(256):
    mllhs_noisy[i] = llh_factorised_noisy(all_X[D_of_size2_ints[i,0]],[0.2]) * llh_factorised_noisy(all_X[D_of_size2_ints[i,1]],[0.2])

In [ ]:
mllhs_factorised = np.zeros(256)
for i in range(256):
    mllhs_factorised[i] = llh_factorised(all_X[D_of_size2_ints[i,0]]) * llh_factorised(all_X[D_of_size2_ints[i,1]])

In [ ]:
# sorted llhs
plt.plot(range(D_of_size2_ints.shape[0]),np.sort(mllhs)[::-1])
plt.plot(range(D_of_size2_ints.shape[0]),np.sort(mllhs_noisy)[::-1])
plt.plot(range(D_of_size2_ints.shape[0]),np.sort(mllhs_factorised)[::-1])
plt.plot(range(D_of_size2_ints.shape[0]),np.sort(mllhs_1x2D)[::-1])
fig = plt.gcf()
fig.set_size_inches(10,3)
# legends
plt.legend(['factorised noisy marginalised', 'factorised fix noise', 'factorised deterministic','1x2D'])
plt.ylabel('mllh')
plt.xlabel('x (sorted by P(x), varies between models)')
plt.title('mllhs for D of size 2')
plt.show()

In [ ]:
randvec = np.random.rand(256)
#randvec = randvec[randvec.argsort()[::-1]]

plt.bar(np.arange(256)[:40], mllhs[:40])
plt.xticks(np.arange(256)[:40], [str(x).replace('.','').replace('[','').replace(']','') for x in all_X_permut[:40]], rotation='vertical')
plt.ylabel('P(x)')
plt.ylim(0,1)
fig = plt.gcf()
fig.set_size_inches(20,4)
plt.show()

In [ ]:
D_of_size2_ints[3,0]

In [ ]:
all_X